In [1]:
import azure.common
import azure.mgmt.containerinstance
import azure.mgmt.resource

print(azure.common.__version__)
print(azure.mgmt.containerinstance.__version__)
print(azure.mgmt.resource.__version__)
# azure-common>=1.1.18
# azure-mgmt-containerinstance>=1.4.1
# azure-mgmt-resource>=2.1.0

from cloudhunky.aci_worker import ACIWorker
from cloudhunky.util import get_afs_creds

1.1.20
1.4.1
2.1.0


In [ ]:
import logging
from pathlib import Path

# logging.basicConfig(level=logging.INFO)

resource_group_name = "SHIP"   
aci_worker = ACIWorker(resource_group_name)

# container_image_name="nvidia/cuda:9.0-base-ubuntu16.04"
container_image_name="mrphys/mfsimage:ff2"
command = ["/bin/bash", "-c", "sleep 28800"]
envs = {"fileName": "pythia8_Geant4_10.0_withCharmandBeauty0_mu.root",
        "mfirstEvent": 0, 
        "nEvents": 100,
        "muShieldDesign":11, 
        "jName": "testJob",
        "jNumber":1}
volume_mount_path = "/sample"
afs_creds = get_afs_creds(Path("/home/igor/LAMBDA/Azure/ekurbatov/afs_creds.yml"))
afs_name = afs_creds["AFS_NAME"]
afs_key = afs_creds["AFS_KEY"]
afs_share = afs_creds["AFS_SHARE"]

container_group_name, logs = aci_worker.run_task_based_container(container_image_name=container_image_name,
                      command=command,
                      cpu=4.0,
                      memory_in_gb=10,
                      gpu_count=0,
                      volume_mount_path=volume_mount_path,
                      envs=envs,
                      timeout=28800,                                          
                      afs_name=afs_name,
                      afs_key=afs_key,
                      afs_share=afs_share,
                      afs_mount_subpath='')

print("Logs for container '{0}':".format(container_group_name))
print("{0}".format(logs.content))

In [5]:
# Launch batch of jobs

In [ ]:
import os
from multiprocessing import Process
import time
import logging


In [ ]:
def run_job(envs):
    resource_group_name = "SHIP"   
    aci_worker = ACIWorker(resource_group_name)
    container_image_name="mrphys/mfsimage:ff2"
    volume_mount_path = "/sample"
    afs_creds = get_afs_creds(Path("/home/igor/LAMBDA/Azure/ekurbatov/afs_creds.yml"))
    afs_name = afs_creds["AFS_NAME"]
    afs_key = afs_creds["AFS_KEY"]
    afs_share = afs_creds["AFS_SHARE"]

    container_group_name, logs = aci_worker.run_task_based_container(container_image_name=container_image_name,
                          command=command,
                          cpu=2.0,
                          memory_in_gb=16,
                          gpu_count=0,
                          volume_mount_path=volume_mount_path,
                          envs=envs,
                          timeout=28800,
                          afs_name=afs_name,
                          afs_key=afs_key,
                          afs_share=afs_share,
                          afs_mount_subpath='')

In [ ]:
procs = []
for i in range(2):
    envs = {"fileName": "pythia8_Geant4_10.0_withCharmandBeauty0_mu.root",
            "mfirstEvent": 0, 
            "nEvents": 100,
            "muShieldDesign":11, 
            "jName": "testJob",
            "jNumber": i}
    proc = Process(target=run_job, args=(envs,))
    procs.append(proc)
    proc.start()
for proc in procs:
    proc .join()